# 1、ChatMessageHistory的使用
场景1: 记忆存储

In [ ]:
from langchain.memory import ChatMessageHistory

# 1、ChatMessageHistory的实例化
history = ChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")

# 3、打印存储的消息
print(history.messages)

场景2: 对接LLM

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
from langchain.memory import ChatMessageHistory # Assuming this is imported

# 1、ChatMessageHistory的实例化
history = ChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1 + 2 * 3 = ?")

# Assuming 'llm' is an instantiated Language Model (e.g., from ChatOpenAI)
response = chat_model.invoke(history.messages)
print(response.content)

# 2、ConversationBufferMemory的使用

举例1: 返回存储的字符串信息

In [ ]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory()

# 2、存储相关的消息
memory.save_context(inputs={"input": "你好, 我叫小明"}, outputs={"output": "很高兴认识你"})
memory.save_context(inputs={"input": "帮我回答一下1+2*3=?"}, outputs={"output": "7"})

# 3、获取存储的信息
print(memory.load_memory_variables({}))

举例2: 以消息列表的方式返回存储的信息

In [ ]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory(return_messages=True)

# 2、存储相关的消息
# inputs对应的是用户消息, outputs对应的是ai消息
memory.save_context(inputs={"human":"你好, 我叫小明"}, outputs={"ai":"很高兴认识你"})
memory.save_context(inputs={"input":"帮我回答一下1+2*3=?"}, outputs={"output":"7"})

# 3、获取存储的信息
print(memory.load_memory_variables({}))

# 说明: 返回的字典结构的key叫history.

print("\n")

# 返回消息列表的方式2:
print(memory.chat_memory.messages)

# 说明: 返回的字典结构的key叫history.

举例3: 结合大模型、提示词模板的使用 (PromptTemplate)

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.prompts import PromptTemplate # Assuming PromptTemplate is imported
from langchain.memory import ConversationBufferMemory # Assuming ConversationBufferMemory is imported
from langchain.chains import LLMChain # Assuming LLMChain is imported

prompt_template = PromptTemplate.from_template(
    template = """
你可以与人类对话。

当前对话历史: {history}

人类问题: {question}

回复:
"""
)

# 3、提供memory实例
memory = ConversationBufferMemory()

# 4、提供Chain
# Assuming 'llm' is an instantiated language model (e.g., ChatOpenAI)
chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)


# # 4、提供Chain
# Assuming LLMChain, llm, prompt_template, and memory are defined from the previous snippet
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"question":"你好, 我的名字叫小明"})
print(response)

In [ ]:
response = chain.invoke({"question":"我的名字是什么"})
print(response)

举例4: 基于举例3, 显式的设置memory的key的值

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.prompts import PromptTemplate # Assuming PromptTemplate is imported
from langchain.memory import ConversationBufferMemory # Assuming ConversationBufferMemory is imported
from langchain.chains import LLMChain # Assuming LLMChain is imported

prompt_template = PromptTemplate.from_template(
    template = """
你可以与人类对话。

当前对话历史: {chatHistory}

人类问题: {question}

回复:
"""
)

# 3、提供memory实例
memory = ConversationBufferMemory(memory_key = "chatHistory")
# 更改变量值

# 4、提供Chain
# Assuming 'llm' is an instantiated language model (e.g., ChatOpenAI)
chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)


# # 4、提供Chain
# Assuming LLMChain, llm, prompt_template, and memory are defined from the previous snippet
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"question":"你好, 我的名字叫小明"})
print(response)

举例5: 结合大模型、提示词模板的使用 (ChatPromptTemplate)

In [ ]:
from langchain_core.messages import SystemMessage # Langchain_core instead of langchain.core.messages
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate, HumanMessagePromptTemplate # Langchain_core instead of langchain.core.prompts
from langchain_openai import ChatOpenAI

# 2. 创建LLM
llm = ChatOpenAI(model_name='gpt-4o-mini')

# 3. 创建Prompt
# Note: The original image uses ("system", "...", MessagesPlaceholder, ("human", "..."))
# ChatPromptTemplate.from_messages handles this structure automatically.
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='history'),
    ("human", "问题: {question}")
])

# 4. 创建Memory
memory = ConversationBufferMemory(return_messages=True)

# 5. 创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

# 6. 调用LLMChain

res1 = llm_chain.invoke({"question": "中国首都都在哪里?"})
print(res1, end="\n\n")

res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)

# 3、ConversationChain的使用
举例1: 以PromptTemplate为例

In [ ]:
from langchain.chains.conversation.base import ConversationChain
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.prompts import PromptTemplate # Assuming PromptTemplate is imported
from langchain.memory import ConversationBufferMemory # Assuming ConversationBufferMemory is imported
from langchain.chains import LLMChain # Assuming LLMChain is imported

prompt_template = PromptTemplate.from_template(
    template = """
你可以与人类对话。

当前对话历史: {history}

人类问题: {input}

回复:
"""
)

# # 3、提供memory实例
# memory = ConversationBufferMemory(memory_key = "chatHistory")
# # 更改变量值
#
# # 4、提供Chain
# # Assuming 'llm' is an instantiated language model (e.g., ChatOpenAI)
# chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

# 3.创建ConversationChain的实列
chain = ConversationChain(llm=chat_model, prompt=prompt_template)

# # 4、提供Chain
# Assuming LLMChain, llm, prompt_template, and memory are defined from the previous snippet
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"input":"你好, 我的名字叫小明"})
print(response)

In [ ]:
response = chain.invoke({"input", "我的名字叫什么"})
print(response)

举例2: 使用默认提供的提示词模板

In [ ]:
from langchain.chains.conversation.base import ConversationChain
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


chat_model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.prompts import PromptTemplate # Assuming PromptTemplate is imported
from langchain.memory import ConversationBufferMemory # Assuming ConversationBufferMemory is imported
from langchain.chains import LLMChain # Assuming LLMChain is imported

# prompt_template = PromptTemplate.from_template(
#     template = """
# 你可以与人类对话。
#
# 当前对话历史: {history}
#
# 人类问题: {input}
#
# 回复:
# """
# )

# # 3、提供memory实例
# memory = ConversationBufferMemory(memory_key = "chatHistory")
# # 更改变量值
#
# # 4、提供Chain
# # Assuming 'llm' is an instantiated language model (e.g., ChatOpenAI)
# chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

# 3.创建ConversationChain的实列
chain = ConversationChain(llm=chat_model)

# # 4、提供Chain
# Assuming LLMChain, llm, prompt_template, and memory are defined from the previous snippet
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"input":"你好, 我的名字叫小明"})
print(response)

# 4、ConversationBufferWindowMemory的使用
举例1:

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2. 实例化ConversationBufferWindowMemory对象, 设定窗口阈值
memory = ConversationBufferWindowMemory(k=2)

# 3. 保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天?"}, {"output": "我不清楚"})

# 4. 读取内存中消息 (返回消息内容的纯文本)
print(memory.load_memory_variables({}))

举例2: 返回消息构成的上下文记忆

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2. 实例化ConversationBufferWindowMemory对象, 设定窗口阈值
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

# 3. 保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话, 你能介绍一下你自己吗?"}, {"output": "当然可以了。我是一个无所不能的小智。"})

# 4. 读取内存中消息 (返回消息内容的纯文本)
print(memory.load_memory_variables({}))

举例3: 结合LLM、chain的使用

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
{history}
Human: {question}
AI:"""
# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)
# 4.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True,
) #
# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么？"})
print(respon4)